In [1]:
# import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET
import re                                                           
from collections import defaultdict
from Stemmer import Stemmer
import time
import os
import gc
from heapq import heappush, heappop
import sys
from math import log10
# from pympler.asizeof import asizeof
import xml.sax

stop_words = set(stopwords.words('english'))
ps = Stemmer("english")

In [2]:
def stopWords(listOfWords):                                         #Stop Words Removal
    temp=[key for key in listOfWords if key not in stop_words]
    return temp


def myTokenizer(text):
    words = re.split(r'(\b[^-\s]+\b)((?<=\.\w).)?', text)
    tok = [i for i in words if i!=None and i != " " and i != ""]
    tok = [ word.lower() for word in tok if re.match('^[a-zA-Z0-9\'-.]+$',word) and not re.match('^[\',-_]+$',word) and not re.match('^[^\w]+$',word)]
    fin_tok = []
    for t in tok:
        fin_tok.append(re.sub("[\+*=&$@/(),.\-!?:]+", '', t))
    fin_tok = [i for i in fin_tok if i!=None and i != " " and i != ""]
    return fin_tok

def processText(text):
    # tokenization
    fin_tok = myTokenizer(text)
    
    # StopWord removal 
    fin_tok = stopWords(fin_tok)
    
    #Stemming
    fin_tok = map(ps.stemWord, fin_tok)
    
    # Final processing
    tok = []
    for t in fin_tok:
        tok.append(re.sub("[']+", '', t))
    return tok

In [3]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [9]:
def findExternalLinks(data):
    if not data:
        return defaultdict(int)
    links = []
    lines = data.split("==external links==")
    if len(lines) > 1:
        lines = lines[1].split("\n")
        for i in xrange(len(lines)):
            if '* [' in lines[i] or '*[' in lines[i]:
                word = ""
                temp = lines[i].split(' ')
                word = [key for key in temp if 'http' not in temp]
                word = ' '.join(word).encode('utf-8')
                links.append(word)
    links = processText(' '.join(links))

    temp = defaultdict(int)
    for key in links:
        temp[key] += 1
    links = temp
    return links

def findReferences(linestr):
    searchStr=linestr.split("==References==")[1:]
    if len(searchStr)<=1:
        return defaultdict(int)
    endInd=searchStr[1].find("==External Links==")
    if endInd==-1:
        endInd=searchStr[1].find("[[Category:")
        if endInd==-1:
            endInd=len(searchStr[1])
    ref = defaultdict(int)
    refs = processText(searchStr[1][:endInd])
    for key in refs:
        ref[key] += 1
    return ref

def findInfoBoxTextCategory(data):     
    if not data:
        return defaultdict(int), defaultdict(int), defaultdict(int)
    info = []
    bodyText = []
    category = []
    links = []
    flagtext = 1
    lines = data.split('\n')
    for i in xrange(len(lines)):
        if '{{infobox' in lines[i]:
            flag = 0
            temp = lines[i].split('{{infobox')[1:]
            info.extend(temp)
            while True:
                if '{{' in lines[i]:
                    count = lines[i].count('{{')
                    flag += count
                if '}}' in lines[i]:
                    count = lines[i].count('}}')
                    flag -= count
                if flag <= 0:
                    break
                i += 1
                info.append(lines[i])

        elif flagtext:
            if '[[category' in lines[i] or '==external links==' in lines[i]:
                flagtext=0
            bodyText.append(lines[i])
            
    else:
        if "[[category" in lines[i]:
            line = data.split("[[category:")
            if len(line)>1:
                category.extend(line[1:-1])
                temp=line[-1].split(']]')
                category.append(temp[0])

    category = processText(' '.join(category))
    info = processText(' '.join(info))
    bodyText = processText(' '.join(bodyText))

    infobox = defaultdict(int)
    for key in info:
        infobox[key] += 1

    bodyTxt = defaultdict(int)
    for key in bodyText:
        bodyTxt[key] += 1

    categ = defaultdict(int)
    for key in category:
        categ[key] += 1
  
    return infobox, bodyTxt, categ

In [10]:
def create_index(title, data):
    twords = processText(title.lower())
    extLinks = findExternalLinks(data.lower())
    refer = findReferences(data.lower())
    if data:
        text = data.replace('_',' ').replace(',','')
    
    ttokens = defaultdict(int)
    for key in twords:
        ttokens[key]+=1
    info, bodyText, category = findInfoBoxTextCategory(text)
    return ttokens, bodyText, info, category, extLinks, refer

In [ ]:
# # dumpPath = 'smallLarge.xml'
# # dumpPath = 'wiki-search-small.xml'
# dumpPath = 'enwiki-latest-pages-articles-multistream.xml'
# pagesPerFile = 18000
# lastCount = 1
# start_time = time.time()
# totalCount = 0
# fileno = 1

# inver = defaultdict(lambda: defaultdict(str))
# ndoc  = defaultdict(lambda: defaultdict(int))

# titMap = open("titleMap.txt","w")

# for event, elem in ET.iterparse(dumpPath, events=('start', 'end')):
#     try:
#         tname = strip_tag_name(elem.tag)
#         if event == 'start':
#             if tname == 'page':
#                 title = ''
#                 did = -1
#                 redirect = ''
#                 inrevision = False
#                 text = ''
#             elif tname == 'revision':
#                 # Do not pick up on revision id's
#                 inrevision = True
#         else:
#             if tname == 'title':
#                 title = elem.text
#             elif tname == 'id' and not inrevision:
#                 did = int(elem.text)
#             elif tname == 'redirect':
#                 redirect = elem.attrib['title']
#             elif tname == 'text':
#                 text = elem.text
#             elif tname == 'page':
#                 totalCount += 1
#                 if totalCount%8000 == 0:
#                     print totalCount
# #                 if totalCount <= 340003:
# #                     elem.clear()
# #                     continue
#                 if redirect == "":
#                     redirect = title
#                 titMap.write(str(did) + ":" + redirect.encode('ascii', 'ignore').decode('ascii')+ "\n")
#                 ttoken, body, tinfo, tcat, extLinks = create_index(redirect,text)
#     #             vocab = list(set(ttoken.keys() + body.keys() + tinfo.keys() + extLinks.keys()))
#     #             inver[keys]["t"] += "|" + str(did)

#                 for keys in ttoken:
#                     inver["t"][keys] += "|" + str(did) + ":" + str(ttoken[keys])
#                     ndoc["t"][keys]  += 1
#                 for keys in body:
#                     inver["b"][keys] += "|" + str(did) + ":" + str(body[keys])
#                     ndoc["b"][keys]  += 1
#                 for keys in tinfo:
#                     inver["i"][keys] += "|" + str(did) + ":" + str(tinfo[keys])
#                     ndoc["i"][keys]  += 1
#                 for keys in tcat:
#                     inver["c"][keys] += "|" + str(did) + ":" + str(tcat[keys])
#                     ndoc["c"][keys]  += 1
#                 for keys in extLinks:
#                     inver["e"][keys] += "|" + str(did) + ":" + str(extLinks[keys])
#                     ndoc["e"][keys]  += 1
#             elem.clear()

#         if totalCount>lastCount and totalCount%pagesPerFile == 0:
#             lastCount = totalCount
#             print fileno, " pages -> ", totalCount
#             for cat in "tbice":
#                 fname = "index/" + cat + "/" + str(fileno) + ".txt"
#                 with open(fname, 'w') as fil:
#                     for key in sorted(inver[cat]):
#                         fil.write(key.encode('ascii', 'ignore').decode('ascii') + inver[cat][key] + "\n")
#             fileno += 1
#             inver.clear()
#             inver = defaultdict(lambda: defaultdict(str))
#             gc.collect()
#     except:
#         continue
        
# for cat in "tbice":
#     fname = "index/" + cat + "/" + str(fileno) + ".txt"
#     with open(fname, 'w') as fil:
#         for key in sorted(inver[cat]):
#             fil.write(key.encode('ascii', 'ignore').decode('ascii') + inver[cat][key] + "\n")

# elapsed_time = time.time() - start_time
# print("Elapsed time: {}".format(hms_string(elapsed_time)))

In [14]:
# dumpPath = 'smallLarge.xml'
# dumpPath = 'wiki-search-small.xml'
dumpPath = 'enwiki-latest-pages-articles-multistream.xml'
pagesPerFile = 10000
lastCount = 1
start_time = time.time()
totalCount = 0
fileno = 1

inver = defaultdict(lambda: defaultdict(str))
ndoc  = defaultdict(lambda: defaultdict(int))

titMap = open("titleMap.txt","w")

class saxHandler( xml.sax.ContentHandler ):
    global pageSize
    
    def __init__(self):
        self.CurrentData = ""
        self.titleFlag = 0
        self.textFlag = 0
        self.idFlag=0
        self.cnt=0
        self.newsitems=[]
        self.news={}


    # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        if tag == 'title':
            self.titleFlag = 1
            self.titleBuff = ""
        elif tag == 'redirect':
            self.titleBuffer = attributes["title"]
        elif tag == 'text':
            self.textFlag = 1
            self.textBuffer = ""
        elif tag == 'id':
            self.idFlag = 1
            self.idBuffer = ""


   # Call when an elements ends
    def endElement(self, tag):
        global inver
        global ndoc
        global fileno
        if tag == 'title':
            self.titleFlag=0
        if tag=='text' and self.textBuffer:
            if self.titleBuffer == "":
                self.titleBuffer = self.titleBuff
            
            titMap.write(str(self.did) + ":" + self.titleBuffer.encode('ascii', 'ignore').decode('ascii')+ "\n")
            ttoken, body, tinfo, tcat, extLinks, refer = create_index(self.titleBuffer, self.textBuffer)
            for keys in ttoken:
                inver["t"][keys] += "|" + str(self.did) + ":" + str(ttoken[keys])
                ndoc["t"][keys]  += 1
            for keys in body:
                inver["b"][keys] += "|" + str(self.did) + ":" + str(body[keys])
                ndoc["b"][keys]  += 1
            for keys in tinfo:
                inver["i"][keys] += "|" + str(self.did) + ":" + str(tinfo[keys])
                ndoc["i"][keys]  += 1
            for keys in tcat:
                inver["c"][keys] += "|" + str(self.did) + ":" + str(tcat[keys])
                ndoc["c"][keys]  += 1
            for keys in extLinks:
                inver["e"][keys] += "|" + str(self.did) + ":" + str(extLinks[keys])
                ndoc["e"][keys]  += 1
            for keys in refer:
                inver["r"][keys] += "|" + str(self.did) + ":" + str(refer[keys])
                ndoc["r"][keys]  += 1
            
            self.textFlag=0
        if tag=='id' and self.idBuffer and self.idFlag==1:
            self.did = int(self.idBuffer)
            self.idFlag+=1
        if tag=='page':
            self.idFlag=0
            self.textFlag=0
            self.titleFlag=0
            self.cnt+=1
            
            if self.cnt%pagesPerFile == 0:
                print fileno, " pages -> ", self.cnt
                for cat in "tbicer":
                    fname = "index/" + cat + "/" + str(fileno) + ".txt"
                    with open(fname, 'w') as fil:
                        for key in sorted(inver[cat]):
                            fil.write(key.encode('ascii', 'ignore').decode('ascii') + inver[cat][key] + "\n")
                fileno += 1
                inver.clear()
                del inver
                inver = defaultdict(lambda: defaultdict(str))
                gc.collect()
                
        if tag=="mediawiki":
            for cat in "tbicer":
                fname = "index/" + cat + "/" + str(fileno) + ".txt"
                with open(fname, 'w') as fil:
                    for key in sorted(inver[cat]):
                        fil.write(key.encode('ascii', 'ignore').decode('ascii') + inver[cat][key] + "\n")
        self.CurrentData = ""


   # Call when a character is read
    def characters(self, content):
        if self.titleFlag:
            self.titleBuff += content
        elif self.textFlag:
            self.textBuffer += content
        elif self.idFlag == 1:
            self.idBuffer += content

#New Index
# create an XMLReader
parser = xml.sax.make_parser()
# turn off namepsaces
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
# override the default ContextHandler
Handler = saxHandler()
parser.setContentHandler( Handler )
parser.parse(dumpPath)  

1  pages ->  1000
2  pages ->  2000
3  pages ->  3000
4  pages ->  4000
5  pages ->  5000


In [4]:
# Code for mutiway merge sort starts


heap = []
fp = []
def pushin(did):
    pline = fp[did].readline().strip(" \n")
    if pline:
        pline = list(pline.partition("|"))
        pline.append(did)
        heappush(heap,pline)
    else:
        print did, " ends"
    

def nextTok():
    cur = heappop(heap)
    tok = cur[0]
    plist = cur[2]
    did = cur[3]
    return tok, plist, did

In [23]:
# for cat in xrange(10):
#     if cat%2==1:
#         continue
#     global heap 
#     global fp
files = ["final_index/t.txt", "final_index/t1.txt" ]
fp = [open(fil) for fil in files]
print fp
for did in xrange(len(fp)):
    pushin(did)
if not heap:
    fp = [f.close() for f in fp]
    exit()

# index = defaultdict(str)
cur, plist, did = nextTok()
pushin(did)

nword, nplist, did = nextTok()
pushin(did)

final = open("final_index/ft.txt", 'w')
last = 'a'
while heap:
    if nword == cur:
        plist += "|" + nplist
    else:
        final.write(cur.encode('ascii', 'ignore').decode('ascii') + "|" + plist + "\n")
#         index[cur] = plist
        cur, plist = nword, nplist
        if 'a' <= cur[0] <= 'z' and cur[0] != last:
            last = cur[0]
            print last
#             for key in sorted(index):
#                 cnt += 1

#             index.clear()
#             index = defaultdict(str)
#             gc.collect()

    nword, nplist, did = nextTok()
    pushin(did)

for f in fp:
    f.close()
final.close()
# cat += 1

[<open file 'final_index/t.txt', mode 'r' at 0x7f370b1a1f60>, <open file 'final_index/t1.txt', mode 'r' at 0x7f370b1a16f0>]
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
0  ends
1  ends


In [28]:
# fil = open('invertedIndex.txt', 'a')
# for key in sorted(index):
#     cnt += 1
#     fil.write(key.encode('ascii', 'ignore').decode('ascii') + "|" + index[key] + "\n")
# fil.close()
def createSecIndex():
    secIndPnt = open("final_index/secInverIndexE.txt", "w")
    lineCnt = 0
    with open("final_index/e.txt", "r") as fil:
        item = fil.readline()
        while item:
            token = item.partition("|")[0]
            secIndPnt.write(token + ":" + str(lineCnt)+"\n")
            lineCnt += len(item.encode('utf-8'))
            item = fil.readline()
            
    secIndPnt.close()
createSecIndex()

In [ ]:
# fp = open("final_index/b.txt", "r")
# secIndPnt = open("final_index/secInverIndex.txt", "r")
# for i in range(10):
#     line = secIndPnt.readline()
#     offset = int(line.split(":")[1].rstrip("\n"))
#     print offset
#     fp.seek(offset, 0)
#     print fp.readline().partition("|")[0]
# fp.close()    